In [98]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NLP").getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [99]:
from pyspark.ml.feature import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# For pipeline development
from pyspark.ml import Pipeline

In [100]:
path = "/home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/"

df=spark.read.csv(path+'kickstarter.csv', inferSchema=True, header=True)

In [101]:
df.limit(4).toPandas()

23/01/25 10:14:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,_c0,blurb,state
0,1,"Using their own character, users go on educati...",failed
1,2,"MicroFly is a quadcopter packed with WiFi, 6 s...",successful
2,3,"A small indie press, run as a collective for a...",failed
3,4,Zylor is a new baby cosplayer! Back this kicks...,failed


In [102]:
df.show(4, False)

23/01/25 10:14:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+---+-----------------------------------------------------------------------------------------------------------------------------------+----------+
|_c0|blurb                                                                                                                              |state     |
+---+-----------------------------------------------------------------------------------------------------------------------------------+----------+
|1  |Using their own character, users go on educational quests around a virtual world leveling up subject-oriented skills (ie Physics). |failed    |
|2  |MicroFly is a quadcopter packed with WiFi, 6 sensors, and 3 p

In [103]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- blurb: string (nullable = true)
 |-- state: string (nullable = true)



In [104]:
df.count()

223627

In [105]:
def null_value_calc(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        if(nullRows>0):
            temp = k,nullRows,(nullRows/numRows)*100
            null_columns_counts.append(temp)
    return(null_columns_counts)

null_columns_calc_list = null_value_calc(df)
spark.createDataFrame(null_columns_calc_list, ['Column_Name', 'Null_Values_Count','Null_Value_Percent']).show()


23/01/25 10:14:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 
 Schema: _c0
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+-----------+-----------------+------------------+
|Column_Name|Null_Values_Count|Null_Value_Percent|
+-----------+-----------------+------------------+
|      blurb|             1488|0.6653937136392296|
|      state|            13157| 5.883457722010312|
+-----------+-----------------+------------------+



In [106]:
df.na.drop().count()

23/01/25 10:14:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


210470

In [107]:
df = df.dropna()

In [108]:
df.count()

23/01/25 10:14:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


210470

In [109]:
df.groupBy("state").count().orderBy(col("count").desc()).show()

23/01/25 10:14:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+--------------------+------+
|               state| count|
+--------------------+------+
|          successful|103582|
|              failed|102000|
| and get some col...|     8|
|          ","failed"|     6|
|     their childhood|     6|
|                love|     6|
| about a lonely f...|     5|
|             romance|     4|
|              poetry|     4|
|            mastered|     4|
|                  CD|     3|
| She Wrote"" but ...|     3|
|               music|     3|
|                NY."|     3|
|              2015."|     3|
|            equality|     3|
|               2014"|     3|
|              2014."|     3|
|             Texas."|     3|
| 

In [110]:
df = df.filter("state IN('successful','failed')")

In [111]:
df.groupBy("state").count().orderBy(col("count").desc()).show()

23/01/25 10:14:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+----------+------+
|     state| count|
+----------+------+
|successful|103582|
|    failed|102000|
+----------+------+



In [112]:
df.select("blurb").show(10, False)

23/01/25 10:14:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+-----------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------+
|Using their own character, users go on educational quests around a virtual world leveling up subject-oriented skills (ie Physics). |
|MicroFly is a quadcopter packed with WiFi, 6 sensors, and 3 processors for ultimate stability -- and fits in the palm of your

In [113]:
df = df.withColumn("blurb", translate(col("blurb"), "/"," ")) \
    .withColumn("blurb", translate(col("blurb"), "("," ")) \
    .withColumn("blurb", translate(col("blurb"), ")"," "))

In [114]:
df.select("blurb").show(10, False)

23/01/25 10:14:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+-----------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------+
|Using their own character, users go on educational quests around a virtual world leveling up subject-oriented skills  ie Physics . |
|MicroFly is a quadcopter packed with WiFi, 6 sensors, and 3 processors for ultimate stability -- and fits in the palm of your

In [115]:
df = df.withColumn("blurb", regexp_replace(col("blurb"), "[^A-Za-z ]+", ""))

In [116]:
df.select("blurb").show(10, False)

23/01/25 10:14:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+-------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------+
|Using their own character users go on educational quests around a virtual world leveling up subjectoriented skills  ie Physics |
|MicroFly is a quadcopter packed with WiFi  sensors and  processors for ultimate stability  and fits in the palm of your hand   |
|A small ind

In [117]:
df = df.withColumn("blurb", regexp_replace(col("blurb"), " +", " "))

In [118]:
df.select("blurb").show(10, False)

23/01/25 10:14:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------+
|Using their own character users go on educational quests around a virtual world leveling up subjectoriented skills ie Physics |
|MicroFly is a quadcopter packed with WiFi sensors and processors for ultimate stability and fits in the palm of your hand     |
|A small indie pr

In [119]:
df = df.withColumn("blurb", lower(col("blurb")))

In [120]:
df.select("blurb").show(10, False)

23/01/25 10:14:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+------------------------------------------------------------------------------------------------------------------------------+
|blurb                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------+
|using their own character users go on educational quests around a virtual world leveling up subjectoriented skills ie physics |
|microfly is a quadcopter packed with wifi sensors and processors for ultimate stability and fits in the palm of your hand     |
|a small indie pr

In [121]:
regex_tokenizer = RegexTokenizer(inputCol = "blurb", outputCol = "words", pattern = "\\W")
raw_words = regex_tokenizer.transform(df)
raw_words.show(2, False)

23/01/25 10:14:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+---+------------------------------------------------------------------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0|blurb                                                                                                                         |state     |words                                                                                                                                            |
+---+-----------------------------------------------------------------------------

In [122]:
raw_words.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- blurb: string (nullable = true)
 |-- state: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [123]:
remover = StopWordsRemover(inputCol="words", outputCol = "filtered")

In [124]:
stopwords = remover.getStopWords()
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [125]:
words_df = remover.transform(raw_words)

In [126]:
words_df.limit(4).toPandas()

23/01/25 10:14:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,_c0,blurb,state,words,filtered
0,1,using their own character users go on educatio...,failed,"[using, their, own, character, users, go, on, ...","[using, character, users, go, educational, que..."
1,2,microfly is a quadcopter packed with wifi sens...,successful,"[microfly, is, a, quadcopter, packed, with, wi...","[microfly, quadcopter, packed, wifi, sensors, ..."
2,3,a small indie press run as a collective for au...,failed,"[a, small, indie, press, run, as, a, collectiv...","[small, indie, press, run, collective, authors..."
3,4,zylor is a new baby cosplayer back this kickst...,failed,"[zylor, is, a, new, baby, cosplayer, back, thi...","[zylor, new, baby, cosplayer, back, kickstarte..."


In [127]:
indexer = StringIndexer(inputCol='state', outputCol='label')
feature_data = indexer.fit(words_df).transform(words_df)

23/01/25 10:14:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


In [128]:
feature_data.limit(4).toPandas()

23/01/25 10:14:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,_c0,blurb,state,words,filtered,label
0,1,using their own character users go on educatio...,failed,"[using, their, own, character, users, go, on, ...","[using, character, users, go, educational, que...",1.0
1,2,microfly is a quadcopter packed with wifi sens...,successful,"[microfly, is, a, quadcopter, packed, with, wi...","[microfly, quadcopter, packed, wifi, sensors, ...",0.0
2,3,a small indie press run as a collective for au...,failed,"[a, small, indie, press, run, as, a, collectiv...","[small, indie, press, run, collective, authors...",1.0
3,4,zylor is a new baby cosplayer back this kickst...,failed,"[zylor, is, a, new, baby, cosplayer, back, thi...","[zylor, new, baby, cosplayer, back, kickstarte...",1.0


In [129]:
############################ BEFORE ###############################
regex_tokenizer = RegexTokenizer(inputCol = "blurb", outputCol = "words", pattern = "\\W")
raw_words = regex_tokenizer.transform(df)

remover = StopWordsRemover(inputCol="words", outputCol = "filtered")
words_df = remover.transform(raw_words)

indexer = StringIndexer(inputCol='state', outputCol='label')
feature_data = indexer.fit(words_df).transform(words_df)

feature_data.show(1, False)

23/01/25 10:14:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
23/01/25 10:14:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+---+------------------------------------------------------------------------------------------------------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------

In [130]:
############################ AFTER ###############################
regex_tokenizer = RegexTokenizer(inputCol = "blurb", outputCol = "words", pattern = "\\W")
# raw_words = regex_tokenizer.transform(df)

remover = StopWordsRemover(inputCol=regex_tokenizer.getOutputCol(), outputCol = "filtered")
# words_df = remover.transform(raw_words)

indexer = StringIndexer(inputCol='state', outputCol='label')
#feature_data = indexer.fit(words_df).transform(words_df)

pipeline = Pipeline(stages=[regex_tokenizer, remover, indexer])
data_prep_pl = pipeline.fit(df)

feature_data = data_prep_pl.transform(df)

feature_data.show(1, False)

23/01/25 10:14:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
23/01/25 10:14:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+---+------------------------------------------------------------------------------------------------------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------

In [131]:
feature_data.limit(5).toPandas()

23/01/25 10:14:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,_c0,blurb,state,words,filtered,label
0,1,using their own character users go on educatio...,failed,"[using, their, own, character, users, go, on, ...","[using, character, users, go, educational, que...",1.0
1,2,microfly is a quadcopter packed with wifi sens...,successful,"[microfly, is, a, quadcopter, packed, with, wi...","[microfly, quadcopter, packed, wifi, sensors, ...",0.0
2,3,a small indie press run as a collective for au...,failed,"[a, small, indie, press, run, as, a, collectiv...","[small, indie, press, run, collective, authors...",1.0
3,4,zylor is a new baby cosplayer back this kickst...,failed,"[zylor, is, a, new, baby, cosplayer, back, thi...","[zylor, new, baby, cosplayer, back, kickstarte...",1.0
4,5,hatoful boyfriend meet skeletons a comedy dati...,failed,"[hatoful, boyfriend, meet, skeletons, a, comed...","[hatoful, boyfriend, meet, skeletons, comedy, ...",1.0


In [132]:
hashingTF = HashingTF(inputCol='filtered', outputCol='rawfeatures', numFeatures=20)
HTFfeaturizedData = hashingTF.transform(feature_data)

In [133]:
HTFfeaturizedData.show(1, False)

23/01/25 10:14:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
+---+------------------------------------------------------------------------------------------------------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------+
|_c0|blurb                                                                                                                         |state |words                             

In [134]:
idf = IDF(inputCol='rawfeatures', outputCol='features')
idfModel = idf.fit(HTFfeaturizedData)
TFIDFfeaturizedData = idfModel.transform(HTFfeaturizedData)
TFIDFfeaturizedData.name = 'TFIDFfeaturizedData'

23/01/25 10:14:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


In [135]:
TFIDFfeaturizedData.limit(4).toPandas()

23/01/25 10:14:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,_c0,blurb,state,words,filtered,label,rawfeatures,features
0,1,using their own character users go on educatio...,failed,"[using, their, own, character, users, go, on, ...","[using, character, users, go, educational, que...",1.0,"(3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, ...","(2.252148827177929, 0.0, 0.8915391572399594, 0..."
1,2,microfly is a quadcopter packed with wifi sens...,successful,"[microfly, is, a, quadcopter, packed, with, wi...","[microfly, quadcopter, packed, wifi, sensors, ...",0.0,"(1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 2.0, 1.0, ...","(0.7507162757259763, 0.9418501584120335, 0.0, ..."
2,3,a small indie press run as a collective for au...,failed,"[a, small, indie, press, run, as, a, collectiv...","[small, indie, press, run, collective, authors...",1.0,"(3.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","(2.252148827177929, 0.0, 0.8915391572399594, 0..."
3,4,zylor is a new baby cosplayer back this kickst...,failed,"[zylor, is, a, new, baby, cosplayer, back, thi...","[zylor, new, baby, cosplayer, back, kickstarte...",1.0,"(2.0, 0.0, 0.0, 1.0, 1.0, 3.0, 0.0, 2.0, 2.0, ...","(1.5014325514519526, 0.0, 0.0, 0.6648665945302..."


In [136]:
HTFfeaturizedData = HTFfeaturizedData.withColumnRenamed("rawfeatures","features")
HTFfeaturizedData.name = 'HTFfeaturizedData'

In [137]:
Word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol='filtered', outputCol='features')
model = Word2Vec.fit(feature_data)

W2VfeaturizedData = model.transform(feature_data)
W2VfeaturizedData.limit(4).toPandas()

23/01/25 10:14:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:14:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:15:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,_c0,blurb,state,words,filtered,label,features
0,1,using their own character users go on educatio...,failed,"[using, their, own, character, users, go, on, ...","[using, character, users, go, educational, que...",1.0,"[0.14038110065406986, -0.24811659927114044, 0...."
1,2,microfly is a quadcopter packed with wifi sens...,successful,"[microfly, is, a, quadcopter, packed, with, wi...","[microfly, quadcopter, packed, wifi, sensors, ...",0.0,"[0.27672377347268845, -0.3168490545146845, -0...."
2,3,a small indie press run as a collective for au...,failed,"[a, small, indie, press, run, as, a, collectiv...","[small, indie, press, run, collective, authors...",1.0,"[0.08995955561598142, 0.28520162504476804, 0.1..."
3,4,zylor is a new baby cosplayer back this kickst...,failed,"[zylor, is, a, new, baby, cosplayer, back, thi...","[zylor, new, baby, cosplayer, back, kickstarte...",1.0,"[0.24012657121888228, 0.25589268415101935, 0.1..."


In [138]:
scaler = MinMaxScaler(inputCol='features', outputCol='scaledFeatures')
scalerModel = scaler.fit(W2VfeaturizedData)
scaled_data = scalerModel.transform(W2VfeaturizedData)
scaled_data.limit(4).toPandas()

23/01/25 10:15:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:15:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,_c0,blurb,state,words,filtered,label,features,scaledFeatures
0,1,using their own character users go on educatio...,failed,"[using, their, own, character, users, go, on, ...","[using, character, users, go, educational, que...",1.0,"[0.14038110065406986, -0.24811659927114044, 0....","[0.5792267598773768, 0.49325837641687337, 0.59..."
1,2,microfly is a quadcopter packed with wifi sens...,successful,"[microfly, is, a, quadcopter, packed, with, wi...","[microfly, quadcopter, packed, wifi, sensors, ...",0.0,"[0.27672377347268845, -0.3168490545146845, -0....","[0.6256666184030814, 0.46945486621258775, 0.53..."
2,3,a small indie press run as a collective for au...,failed,"[a, small, indie, press, run, as, a, collectiv...","[small, indie, press, run, collective, authors...",1.0,"[0.08995955561598142, 0.28520162504476804, 0.1...","[0.5620526105827438, 0.6779578135263551, 0.657..."
3,4,zylor is a new baby cosplayer back this kickst...,failed,"[zylor, is, a, new, baby, cosplayer, back, thi...","[zylor, new, baby, cosplayer, back, kickstarte...",1.0,"[0.24012657121888228, 0.25589268415101935, 0.1...","[0.6132011968298638, 0.6678075043140753, 0.639..."


In [139]:
W2VfeaturizedData = scaled_data.select('state','blurb','label','scaledFeatures')
W2VfeaturizedData = W2VfeaturizedData.withColumnRenamed('scaledFeatures','features')
W2VfeaturizedData.name = 'W2VfeaturizedData'

In [140]:
W2VfeaturizedData.limit(3).toPandas()

23/01/25 10:15:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


,state,blurb,label,features
0,failed,using their own character users go on educatio...,1.0,"[0.5792267598773768, 0.49325837641687337, 0.59..."
1,successful,microfly is a quadcopter packed with wifi sens...,0.0,"[0.6256666184030814, 0.46945486621258775, 0.53..."
2,failed,a small indie press run as a collective for au...,1.0,"[0.5620526105827438, 0.6779578135263551, 0.657..."


In [141]:
def ClassTrainEval(classifier,features,classes,train,test):

    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,train):
        
        if Mtype == "OneVsRest":
            # instantiate the base classifier.
            lr = LogisticRegression()
            # instantiate the One Vs Rest Classifier.
            OVRclassifier = OneVsRest(classifier=lr)
#             fitModel = OVRclassifier.fit(train)
            # Add parameters of your choice here:
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1, 0.01]) \
                .build()
            #Cross Validator requires the following parameters:
            crossval = CrossValidator(estimator=OVRclassifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=2) # 3 is best practice
            # Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
        if Mtype == "MultilayerPerceptronClassifier":
            # specify layers for the neural network:
            # input layer of size features, two intermediate of features+1 and same size as features
            # and output of size number of classes
            # Note: crossvalidator cannot be used here
            features_count = len(features[0][0])
            layers = [features_count, features_count+1, features_count, classes]
            MPC_classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
            fitModel = MPC_classifier.fit(train)
            return fitModel
        if Mtype in("LinearSVC","GBTClassifier") and classes != 2: # These classifiers currently only accept binary classification
            print(Mtype," could not be used because PySpark currently only accepts binary classification data for this algorithm")
            return
        if Mtype in("LogisticRegression","NaiveBayes","RandomForestClassifier","GBTClassifier","LinearSVC","DecisionTreeClassifier"):
  
            # Add parameters of your choice here:
            if Mtype in("LogisticRegression"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .addGrid(classifier.maxIter, [10, 15,20])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("NaiveBayes"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6]) \
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("RandomForestClassifier"):
                paramGrid = (ParamGridBuilder() \
                               .addGrid(classifier.maxDepth, [2, 5, 10])
#                                .addGrid(classifier.maxBins, [5, 10, 20])
#                                .addGrid(classifier.numTrees, [5, 20, 50])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("GBTClassifier"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
#                              .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .addGrid(classifier.maxIter, [10, 15,50,100])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("LinearSVC"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxIter, [10, 15]) \
                             .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .build())
            
            # Add parameters of your choice here:
            if Mtype in("DecisionTreeClassifier"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                             .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .build())
            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=2) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        
        if Mtype in("OneVsRest"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            # Extract list of binary models
            models = BestModel.models
            for model in models:
                print('\033[1m' + 'Intercept: '+ '\033[0m',model.intercept,'\033[1m' + '\nCoefficients:'+ '\033[0m',model.coefficients)

        if Mtype == "MultilayerPerceptronClassifier":
            print("")
            print('\033[1m' + Mtype," Weights"+ '\033[0m')
            print('\033[1m' + "Model Weights: "+ '\033[0m',fitModel.weights.size)
            print("")

        if Mtype in("DecisionTreeClassifier", "GBTClassifier","RandomForestClassifier"):
            # FEATURE IMPORTANCES
            # Estimate of the importance of each feature.
            # Each feature’s importance is the average of its importance across all trees 
            # in the ensemble The importance vector is normalized to sum to 1. 
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Feature Importances"+ '\033[0m')
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            print(BestModel.featureImportances)
            
            if Mtype in("DecisionTreeClassifier"):
                global DT_featureimportances
                DT_featureimportances = BestModel.featureImportances.toArray()
                global DT_BestModel
                DT_BestModel = BestModel
            if Mtype in("GBTClassifier"):
                global GBT_featureimportances
                GBT_featureimportances = BestModel.featureImportances.toArray()
                global GBT_BestModel
                GBT_BestModel = BestModel
            if Mtype in("RandomForestClassifier"):
                global RF_featureimportances
                RF_featureimportances = BestModel.featureImportances.toArray()
                global RF_BestModel
                RF_BestModel = BestModel

        if Mtype in("LogisticRegression"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Coefficient Matrix"+ '\033[0m')
            print("You should compares these relative to eachother")
            print("Coefficients: \n" + str(BestModel.coefficientMatrix))
            print("Intercept: " + str(BestModel.interceptVector))
            global LR_coefficients
            LR_coefficients = BestModel.coefficientMatrix.toArray()
            global LR_BestModel
            LR_BestModel = BestModel

        if Mtype in("LinearSVC"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")
            print("Coefficients: \n" + str(BestModel.coefficients))
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

In [142]:
# from pyspark.ml.classification import *
# from pyspark.ml.evaluation import *
# from pyspark.sql import functions
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Comment out Naive Bayes if your data still contains negative values
classifiers = [
                LogisticRegression()
                ,OneVsRest()
               ,LinearSVC()
               ,NaiveBayes()
               ,RandomForestClassifier()
               ,GBTClassifier()
               ,DecisionTreeClassifier()
               ,MultilayerPerceptronClassifier()
              ] 

featureDF_list = [HTFfeaturizedData,TFIDFfeaturizedData,W2VfeaturizedData]

In [143]:
for featureDF in featureDF_list:
    print(featureDF.name)
    train, test = featureDF.randomSplit([0.7,0.3], seed=11)

    features = featureDF.select(['features']).collect()
    class_count = featureDF.select(countDistinct("label")).collect()
    classes = class_count[0][0]

    columns = ['Classifier','Result']
    vals = [("Place Holder", "N/A")]
    results = spark.createDataFrame(vals, columns)

    for classifier in classifiers:
        new_result = ClassTrainEval(classifier, features, classes, train, test)
        results = results.union(new_result)
    results = results.where("Classifier!='Place Holder'")
    print(results.show(truncate=False))

HTFfeaturizedData
23/01/25 10:15:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:15:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
23/01/25 10:15:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:15:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:15:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:15:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


...

23/01/25 10:32:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


 
DecisionTreeClassifier  Feature Importances
(Scores add up to 1)
Lowest score is the least important
 
(3,[0,1,2],[0.3912169716714286,0.5538695637905796,0.05491346453799188])
23/01/25 10:32:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:32:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv



MultilayerPerceptronClassifier  Weights
Model Weights:  39

23/01/25 10:32:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


+------------------------------+------+
|Classifier                    |Result|
+------------------------------+------+
|LogisticRegression            |55.33 |
|OneVsRest                     |55.37 |
|LinearSVC                     |54.48 |
|NaiveBayes                    |50.42 |
|RandomForestClassifier        |57.19 |
|GBTClassifier                 |57.12 |
|DecisionTreeClassifier        |56.95 |
|MultilayerPerceptronClassifier|57.08 |
+------------------------------+------+

None


In [144]:
classifier = RandomForestClassifier()
featureDF = W2VfeaturizedData

train, test = featureDF.randomSplit([0.7, 0.3], seed=11)
features = featureDF.select(['features']).collect()

class_count = featureDF.select(countDistinct('label')).collect()
classes = class_count[0] [0]

ClassTrainEval(classifier, features, classes, train, test)

23/01/25 10:44:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:44:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv
23/01/25 10:44:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:44:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:44:26 WARN DAGScheduler: Broadcasting large task binary with size 1287.8 KiB
23/01/25 10:44:27 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
23/01/25 10:44:28 WARN DAGScheduler: Broadcasting large task binary with size 1422.4 KiB
23/01/25 10:44:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:44:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


...

23/01/25 10:44:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:44:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


23/01/25 10:44:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


 
RandomForestClassifier  Feature Importances
(Scores add up to 1)
Lowest score is the least important
 
(3,[0,1,2],[0.38750004546505934,0.528986547761474,0.08351340677346668])
23/01/25 10:44:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


DataFrame[Classifier: string, Result: string]

In [147]:
predictions = RF_BestModel.transform(test)
print("Predicted Failures:")
predictions.select("state","blurb").filter("prediction=0").orderBy(predictions["prediction"].desc()).show(3,False)
print(" ")
print("Predicted Success:")
predictions.select("state","blurb").filter("prediction=1").orderBy(predictions["prediction"].desc()).show(3,False)

Predicted Failures:
23/01/25 10:51:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , blurb, state
 Schema: _c0, blurb, state
Expected: _c0 but found: 
CSV file: file:///home/randi/Documents/pyspark_udemy/Jupyter+Notebooks+and+Datasets+AS+of+16MAY21/Jupyter_Notebooks_and_Datasets_AS_of_16MAY21/Practice/Datasets/kickstarter.csv


+------+--------------------------------------------------------------------------------------------------------------------------------+
|state |blurb                                                                                                                           |
+------+--------------------------------------------------------------------------------------------------------------------------------+
|failed|a san francisco based blog where artists from all over can come together to find inspiration as well as support each other      |
|failed|aspiring canadian music producer and singersong writer who lives and loves music and art creating a debut alternative hip hop ep|
|failed|everyday for the next days i will be creating a new digital portrait ive done so far the feedbacks been absolutely amazing      |
+------+--------------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows

 
Predict

+------+-------------------------------------------------------------------------------------------------------------------------------+
|state |blurb                                                                                                                          |
+------+-------------------------------------------------------------------------------------------------------------------------------+
|failed| one pen stroke at a time                                                                                                      |
|failed|a pilot program of five students focused on the interactions between ecology and humanity in the ceramic village of tamba japan|
|failed|a public art studio embracing various fine arts and encouraging cultural crafts experienced students challenged art craft shows|
+------+-------------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [148]:
spark.stop()